# **SIIM-FISABIO-RSNA COVID-19 Detection**

**Task:** To identify and localize COVID-19 abnormalities on chest radiographs

# **Tensorflow Records**

A TFRecord file stores your data as a sequence of binary strings. Binary data takes up less space on disk, takes less time to copy and can be read much more efficiently from disk. However, pure performance isn’t the only advantage of the TFRecord file format. It is optimized for use with Tensorflow in multiple ways. One use being, it makes it easy to combine multiple datasets and integrates seamlessly with the data import and preprocessing functionality provided by the library. Especially for datasets that are too large to be stored fully in memory this is an advantage as only the data that is required at the time (e.g. a batch) is loaded from disk and then processed. 

# **Loading Packages**

In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
import sys
import pydicom
from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import hashlib
import os
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
import cv2
from tqdm.auto import tqdm
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline

# **Reading csv file**

Here I'm using an updated csv file not the original file provided in the competition.

In [ ]:
raw_df = pd.read_csv('../input/updated-csv/combined_train_data.csv')
raw_df.class_id = raw_df.class_id - 1
raw_df.head()



# **Helpers Functions**

In [ ]:
IMAGE_SIZE = 1024 # change this to desired value
CLIP_LIMIT = 2.
GRID_SIZE = (8,8)

NUM_CLASSES = 3

# https://sashat.me/2017/01/11/list-of-20-simple-distinct-colors/
LABEL_COLORS = [(230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200)]

def read_image(path, voi_lut = True, target_size=IMAGE_SIZE, use_clahe=True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    ds = dcmread(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(ds.pixel_array, ds)
    else:
        data = ds.pixel_array
    im = data - np.min(data)
    im = 255. * im / np.max(im)
    if ds.PhotometricInterpretation == "MONOCHROME1": # check for inverted image
        im = 255. - im
    if use_clahe:
        clahe = cv2.createCLAHE(clipLimit=CLIP_LIMIT, tileGridSize=GRID_SIZE)
        climg = clahe.apply(im.astype('uint8'))
        img = Image.fromarray(climg.astype('uint8'), 'L')
    else:
        img = Image.fromarray(im.astype('uint8'), 'L')
    org_size = img.size
    if max(img.size) > target_size:
        img.thumbnail((target_size, target_size), Image.ANTIALIAS)
    return img, org_size

In [ ]:
findings = raw_df[raw_df.class_id != -1]
class_names = []
for i in range(NUM_CLASSES):
    class_names.append(findings[findings.class_id == i].class_name.iloc[0])
class_names

In [ ]:
wbf = []

for _, row in tqdm(findings.iterrows()):
    image_name = row['id'].replace('_image','')        
    label = row['label'].split()
    length = row['No_of_findings']
    class_id = row['class_id']
    class_name = row['class_name']
    path = row['img_path']
    
    j = 0
    for i in range(length):
        x_min = float(label[j+2])
        y_min = float(label[j+3])            
        x_max = float(label[j+4])           
        y_max = float(label[j+5])
        # Hard code below class_name and class_id to opacity and 0 resp. if you want to train on 2-class objects (disease or no disease)
        #
        wbf.append([image_name, class_name, class_id, x_min, y_min, x_max, y_max, path])
        j += 6
    
wbf_df = pd.DataFrame (wbf, columns=['image_id', 'class_name', 'class_id', 'x_min', 'y_min', 'x_max', 'y_max', 'path'])
wbf_df.to_csv('wbf_objects.csv')

In [ ]:
wbf_df.head()

In [ ]:
wbf_df.class_id.value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold

NUM_SHARDS = 20

skf = StratifiedKFold(n_splits=NUM_SHARDS, shuffle=True, random_state=42)
df_folds = wbf_df[['image_id']].copy()

df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'object_count'] = wbf_df.groupby('image_id')['class_id'].nunique()

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['object_count'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str))

df_folds.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
df_folds.reset_index(inplace=True)

In [ ]:
df_folds

In [ ]:
df_shard = pd.merge(wbf_df, df_folds[df_folds['fold'] == 0], on='image_id')
dfs = df_shard.class_name.value_counts().to_frame('S0').sort_index()
for i in range(1,20):
    df_shard = pd.merge(wbf_df, df_folds[df_folds['fold'] == i], on='image_id')
    dfs['S'+str(i)] = df_shard.class_name.value_counts().to_frame().sort_index()
dfs

In [ ]:
# Create example for TensorFlow Object Detection API
def create_tf_example(imagedf, longest_edge=IMAGE_SIZE):  
    fname = imagedf.path.iloc[0]
    filename=fname.split('/')[-1] # exclude path    
    img, org_size = read_image(fname, target_size=IMAGE_SIZE, use_clahe=True)
    height = img.size[1] # Image height
    width = img.size[0] # Image width
    buf= BytesIO()
    img.save(buf, format= 'JPEG') # encode to jpeg in memory
    encoded_image_data= buf.getvalue()
    image_format = b'jpeg'
    source_id = imagedf.image_id.iloc[0]
    # A hash of the image is used in some frameworks
    key = hashlib.sha256(encoded_image_data).hexdigest()   
    # object bounding boxes 
    xmins = imagedf.x_min.values/org_size[0] # List of normalized left x coordinates in bounding box 
    xmaxs = imagedf.x_max.values/org_size[0] # List of normalized right x coordinates in bounding box
    ymins = imagedf.y_min.values/org_size[1] # List of normalized top y coordinates in bounding box 
    ymaxs = imagedf.y_max.values/org_size[1] # List of normalized bottom y coordinates in bounding box
    # List of string class name & id of bounding box (1 per box)
    object_cnt = len(imagedf)
    classes_text = []
    classes = []
    for i in range(object_cnt):
        classes_text.append(imagedf.class_name.iloc[i].encode())
        classes.append(1+imagedf.class_id.iloc[i]) # 0 is not a valid class
        
    # unused features from Open Image 
    depiction = np.zeros(object_cnt, dtype=int)
    group_of = np.zeros(object_cnt, dtype=int)
    occluded = np.zeros(object_cnt, dtype=int) #also Pascal VOC
    truncated = np.zeros(object_cnt, dtype=int) # also Pascal VOC
    # Pascal VOC
    view_text = []
    for i in range(object_cnt):
        view_text.append('frontal'.encode())
    difficult = np.zeros(object_cnt, dtype=int)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode()])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'image/object/depiction': tf.train.Feature(int64_list=tf.train.Int64List(value=depiction)),
        'image/object/group_of': tf.train.Feature(int64_list=tf.train.Int64List(value=group_of)),
        'image/object/occluded': tf.train.Feature(int64_list=tf.train.Int64List(value=occluded)),
        'image/object/truncated': tf.train.Feature(int64_list=tf.train.Int64List(value=truncated)),
        'image/object/difficult': tf.train.Feature(int64_list=tf.train.Int64List(value=difficult)),
        'image/object/view': tf.train.Feature(bytes_list=tf.train.BytesList(value=view_text))
    }))
    return tf_record

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import contextlib2

def open_sharded_tfrecords(exit_stack, base_path, num_shards):
    tf_record_output_filenames = [
        '{}-{:03d}-of-{:03}.tfrecord'.format(base_path, idx, num_shards)
        for idx in range(num_shards)
        ]
    tfrecords = [
        exit_stack.enter_context(tf.io.TFRecordWriter(file_name))
        for file_name in tf_record_output_filenames
    ]
    return tfrecords

output_filebase='./SiimCovid19'

img_cnt = np.zeros(NUM_SHARDS, dtype=int)
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_tfrecords(tf_record_close_stack, output_filebase, NUM_SHARDS)
    for i in tqdm(range(NUM_SHARDS)):
        df_shard = pd.merge(wbf_df, df_folds[df_folds['fold'] == i], on='image_id')
        ids = df_shard.image_id.unique()
        for j in range (len(ids)):
            imagedf = df_shard[df_shard.image_id == ids[j]]
            tf_record = create_tf_example(imagedf, longest_edge=IMAGE_SIZE)            
            output_tfrecords[i].write(tf_record.SerializeToString())
            img_cnt[i] += 1
print("Converted {} images".format(np.sum(img_cnt)))
print("Images per shard: {}".format(img_cnt))

# **Exporting the TFRecords**

In [ ]:
%%time
!tar -zcf siim_data.tar.gz -C "/kaggle/working/" .

Download it from the output folder

In [ ]:
import json

dparams = {
    "IMAGE_SIZE": IMAGE_SIZE,
    "CLIP_LIMIT": CLIP_LIMIT,
    "GRID_SIZE": GRID_SIZE}
with open("dparams.json", "w") as json_file:
    json_file.write(json.dumps(dparams, indent = 4))

In [ ]:
labels = ['Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']

with open('./SiimCovid19.pbtxt', 'w') as f:
    for i in range (len(labels)): 
        f.write('item {{\n id: {}\n name:\'{}\'\n}}\n\n'.format(i+1, labels[i]))

Download these files too (json.dumps and the pbtxt file) for training purpose

# Checking TFRecords

In [ ]:
# Some helper functions to draw image with object boundary boxes
fontname = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
font = ImageFont.truetype(fontname, 40) if os.path.isfile(fontname) else ImageFont.load_default()

def bbox(img, xmin, ymin, xmax, ymax, color, width, label, score):
    draw = ImageDraw.Draw(img)
    xres, yres = img.size[0], img.size[1]
    box = np.multiply([xmin, ymin, xmax, ymax], [xres, yres, xres, yres]).astype(int).tolist()
    txt = " {}: {}%" if score >= 0. else " {}"
    txt = txt.format(label, round(score, 1))
    ts = draw.textsize(txt, font=font)
    draw.rectangle(box, outline=color, width=width)
    if len(label) > 0:
        if box[1] >= ts[1]+3:
            xsmin, ysmin = box[0], box[1]-ts[1]-3
            xsmax, ysmax = box[0]+ts[0]+2, box[1]
        else:
            xsmin, ysmin = box[0], box[3]
            xsmax, ysmax = box[0]+ts[0]+2, box[3]+ts[1]+1
        draw.rectangle([xsmin, ysmin, xsmax, ysmax], fill=color)
        draw.text((xsmin, ysmin), txt, font=font, fill='white')

def plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by):
    img = img.convert("RGB")
    for i in range(len(xmin)):
        color = LABEL_COLORS[class_label[i]]
        bbox(img, xmin[i], ymin[i], xmax[i], ymax[i], color, 5, classes[i].decode(), -1)
    plt.setp(axes, xticks=[], yticks=[])
    axes.set_title(by)
    plt.imshow(img)

In [ ]:
fname='./SiimCovid19-018-of-020.tfrecord' 
dataset3 = tf.data.TFRecordDataset(fname)
fig = plt.figure(figsize=(20,30))
idx=1
for raw_record in dataset3.take(6):
    axes = fig.add_subplot(3, 2, idx)
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    xmin=example.features.feature['image/object/bbox/xmin'].float_list.value[:]
    xmax=example.features.feature['image/object/bbox/xmax'].float_list.value[:]
    ymin=example.features.feature['image/object/bbox/ymin'].float_list.value[:]
    ymax=example.features.feature['image/object/bbox/ymax'].float_list.value[:]
    classes=example.features.feature['image/object/class/text'].bytes_list.value[:]
    class_label=example.features.feature['image/object/class/label'].int64_list.value[:]
    img_encoded=example.features.feature['image/encoded'].bytes_list.value[0]
    img = Image.open(BytesIO(img_encoded))
    plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, "")
    idx=idx+1

# Credits

Copied from the below mentioned notebook and modified according to the given data for this competition

https://www.kaggle.com/mistag/data-create-tfrecords-of-vinbigdata-chest-x-rays

Thankyou so much sir.